# Distance to training samples

Alessandro Samuel-Rosa, Taciara Zborowski Horst

2025-06-29

This script computes the distance from each pixel in a raster to the nearest training sample point. If the nearest point is further than 1000 meters, the distance is set to 1000 meters. The script uses the Google Earth Engine (GEE) API to perform these calculations. The fast distance calculation is done using the `ee.Image.fastDistanceTransform` method, which is efficient for large datasets. The function outputs distance values in number of pixels, which can be converted to meters based on the scale of the raster.

License: MIT

In [1]:
# Import necessary libraries
import ee
import geemap

# Initialize the Earth Engine API
# ee.Authenticate()
ee.Initialize(project='mapbiomas-solos-workspace')

# --- Configuration ---
# Asset IDs
aoi_path = 'projects/mapbiomas-workspace/AUXILIAR/brasil_2km'
fc_dir = 'projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/granulometry/'
fc_name = 'matriz-v010_0_10cm'
fc_path = f'{fc_dir}{fc_name}'

# Parameters for distance calculation and visualization
# Use a neighborhood large enough to find distances > 1000m before we mask them.
NEIGHBORHOOD_PIXELS = 5000
# The distance in meters at which we want to cap the result.
MAX_DISTANCE_METERS_THRESHOLD = 1000

print("--- Calculating and simplifying meter-distance, masking > 1000m ---")

--- Calculating and simplifying meter-distance, masking > 1000m ---


In [2]:
# --- 1. Define Area of Interest ---
print('Loading AOI:', aoi_path)
aoi_asset = ee.FeatureCollection(aoi_path)
aoi_geometry = aoi_asset.union(1)
print("AOI geometry loaded.")


Loading AOI: projects/mapbiomas-workspace/AUXILIAR/brasil_2km
AOI geometry loaded.


In [3]:
# --- 2. Load and Filter Soil Samples ---
print('Loading and filtering soil samples...')
soil_points_all = ee.FeatureCollection(fc_path)
soil_points = soil_points_all.filterBounds(aoi_geometry)
print("Soil samples loaded.")

Loading and filtering soil samples...
Soil samples loaded.


In [4]:
# --- 3. Add constant property ---
def add_constant(feat):
  return feat.set({'constant': 1})
soil_points_with_constant = soil_points.map(add_constant)

In [5]:
# --- 4. Prepare Image for Distance Calculation (No Reprojection) ---
print("\nRasterizing points (using default projection/scale)...")

# Reduce to Image - uses GEE default projection/scale
point_presence_image = soil_points_with_constant.reduceToImage(
    properties=['constant'],
    reducer=ee.Reducer.sum().unweighted()
).unmask(0)

# Create the source image (1 where points are, 0 elsewhere)
source_image = point_presence_image.gt(0)


Rasterizing points (using default projection/scale)...


In [6]:
# --- 5. Calculate Distance in Pixels ---
print("Calculating distance in PIXELS using fastDistanceTransform...")
distance_image_multiband = source_image.fastDistanceTransform(
    neighborhood=NEIGHBORHOOD_PIXELS,
    metric='squared_euclidean'
)
distance_pixels_image = distance_image_multiband.select('distance').sqrt()

Calculating distance in PIXELS using fastDistanceTransform...


In [7]:
# --- 6. Convert to Approximate Meters ---
print("Converting pixel distance to approximate METERS using pixelArea()...")
pixel_size_in_meters = ee.Image.pixelArea().sqrt()
distance_meters_image = distance_pixels_image.multiply(pixel_size_in_meters)

Converting pixel distance to approximate METERS using pixelArea()...


In [8]:
# --- 7. Mask distances greater than the threshold ---
print(f"Masking out all distances > {MAX_DISTANCE_METERS_THRESHOLD} meters...")
# The .lte() function means "less than or equal to".
# The mask keeps only the pixels that meet this condition.
final_distance_image = distance_meters_image.updateMask(
    distance_meters_image.lte(MAX_DISTANCE_METERS_THRESHOLD)
)

Masking out all distances > 1000 meters...


In [9]:
# --- 8. Simplify the final image ---
print("Simplifying final image by dividing by 10, rounding, and converting to integer...")
# This results in an image with values from 0-100. toUint8 is efficient for this range.
simplified_distance_image = final_distance_image.divide(10).round().toUint8()


Simplifying final image by dividing by 10, rounding, and converting to integer...


In [10]:
# --- 9. Clip Result ---
print("\nClipping final simplified image to AOI...")
final_simplified_image_clipped = simplified_distance_image.clip(aoi_geometry)


Clipping final simplified image to AOI...


In [11]:
# --- 10. Visualize the final simplified Map ---
print("\nPreparing map visualization...")
Map = geemap.Map()
Map.centerObject(aoi_geometry, 4)

# --- UPDATED: Visualization for the new 0-100 range ---
dnn_vis_params = {
  'min': 0,
  'max': 100, # Corresponds to the original 1000m max distance
  'palette': ['0000FF', '00FFFF', 'FFFF00', 'FF0000'] # Blue=Near -> Red=Far
}
print(f"Visualizing simplified distances from 0 to 100.")

# Add layers to the map
Map.addLayer(ee.Image().paint(aoi_geometry, 0, 2), {'palette': '000000'}, 'AOI Boundary')
Map.addLayer(soil_points, {'color': 'FF0000'}, 'Filtered Soil Samples (Red)')
Map.addLayer(
    final_simplified_image_clipped,
    dnn_vis_params,
    'Simplified Distance (0-100)'
)

print("\nScript finished. Displaying map.")
Map


Preparing map visualization...
Visualizing simplified distances from 0 to 100.

Script finished. Displaying map.


Map(center=[-10.62347807211283, -53.19353084531423], controls=(WidgetControl(options=['position', 'transparent…